In [1]:
# Set up data collection foundation

CREATURE_NAME=[]
CREATURE_CAT=[]
SEASON_EP=[]
SHOW_NAME=[]

TABLE_COLUMNS = ['creature_name', 'creature_cat', 'season_ep', 'show_name']

DATABASE = {'creature_name':CREATURE_NAME, 'creature_cat':CREATURE_CAT, 'season_ep':SEASON_EP, 'show_name':SHOW_NAME}

MERLIN_URL = 'http://merlin.wikia.com/wiki/Creatures_of_Magic'

In [2]:
# Libraries
from bs4 import BeautifulSoup as soup
import pandas as pd
import re
from urllib import request

In [3]:
# Convert html to a more user friendly format
def soupTheLink(url):
    html = request.urlopen(url).read().decode('utf8')
    return soup(html, 'lxml')

In [4]:
# Scrape a list of links to the categories of creatures
def collectCatLinks(url):

    catLinks = []

    souped = soupTheLink(url)

    body = souped.find(id='mw-content-text')
    header = body.find(id='Known_Creatures_of_Magic').find_parent()
    categories = header.find_next_sibling('ul')
    for category in categories.find_all('li'):
        link = category.find('a', href = True)
        try:
            fullLink = 'http://merlin.wikia.com' + link['href']
            catLinks.append(fullLink)
        except TypeError:
            pass
    return catLinks

In [6]:
# Jump into the list of links to the creatures based on the categories
def intoCategoryLinks(url):
    print('categoryLink: ', url)
    souped = soupTheLink(url)

    body = souped.find(id='mw-content-text')
    table = body.find('table')
    headline = body.find(id='Appearances')
    if headline == None and table != None:
        pageLinks = table.find('tr')

        for creature in pageLinks.find_next_siblings('tr'):
            for links in creature.find_all('td', limit=1):
                try:
                    link = links.find('a', href = True)                    
                    fullLink = 'http://merlin.wikia.com' + link['href']
                    intoCreatureLink(fullLink)
                except TypeError:
                    pass
                
    elif headline != None:
        intoCreatureLink(url)   

In [7]:
# Import the rest of the data to the database
def intoCreatureLink(creature):

    print('creatureLink: ', creature)
    souped = soupTheLink(creature)

    #collect creature name
    name = creature[29:]
    DATABASE['creature_name'].append(name)

    #collect show name
    DATABASE['show_name'].append('Merlin')

    #collect season appearances
    currentSeason = 0
    seasonLink = ''
    body = souped.find(id='mw-content-text')
    SEASON_EP=[]
    headline = body.find(id='Appearances')
    if headline != None:

        headline = headline.find_parent()
        seasonList = headline.find_next_sibling('dl')
        for seasons in seasonList.find_all('dt'):
            seasonDegree=[]
            seasonText = seasons.get_text()
            try:
                season = int(seasonText[7])
            except ValueError:
                season = 'None'
            degree = 0
            for episode in seasons.find_next_siblings('dd'):
                degree += 1
            seasonDegree.extend([season, degree])
        DATABASE['season_ep'].append(seasonDegree) 
    else:
        DATABASE['season_ep'].append('None') 

In [8]:
# Main program
def collectCreatureLinks(catLinks):
    for mainLink in catLinks:
        intoCategoryLinks(mainLink)

In [9]:
# Run program
categoryLinks = collectCatLinks(MERLIN_URL)
collectCreatureLinks(categoryLinks)
print('Database: ', DATABASE)

categoryLink:  http://merlin.wikia.com/wiki/Warlock
creatureLink:  http://merlin.wikia.com/wiki/Merlin
creatureLink:  http://merlin.wikia.com/wiki/Ghost_Boy
creatureLink:  http://merlin.wikia.com/wiki/Gaius
creatureLink:  http://merlin.wikia.com/wiki/Edwin_Muirden
creatureLink:  http://merlin.wikia.com/wiki/Gilli
creatureLink:  http://merlin.wikia.com/wiki/Druids
creatureLink:  http://merlin.wikia.com/wiki/Druid_Boy
creatureLink:  http://merlin.wikia.com/wiki/Mordred
creatureLink:  http://merlin.wikia.com/wiki/Balinor
creatureLink:  http://merlin.wikia.com/wiki/Alator
creatureLink:  http://merlin.wikia.com/wiki/Taliesin
creatureLink:  http://merlin.wikia.com/wiki/Tauren
creatureLink:  http://merlin.wikia.com/wiki/Alvarr
creatureLink:  http://merlin.wikia.com/wiki/Anhora
creatureLink:  http://merlin.wikia.com/wiki/Gregor
categoryLink:  http://merlin.wikia.com/wiki/Witch
creatureLink:  http://merlin.wikia.com/wiki/Morgana_Pendragon
creatureLink:  http://merlin.wikia.com/wiki/Morgause
cre

In [15]:
CREATURE_CAT = ['Warlock', 'Warlock', 'Warlock', 'Warlock', 'Warlock', 'Warlock', 'Warlock', 'Warlock', 'Warlock', 'Warlock', 'Warlock', 'Warlock', 'Warlock', 'Warlock', 'Warlock', 'Witch', 'Witch', 'Witch', 'Witch', 'Witch', 'Witch', 'Witch', 'Witch', 'Druid', 'Seer', 'Seer', 'Seer', 'Seer', 'Seer', 'Seer'
, 'Seer', 'Dochraid', 'Dragon', 'Fomorroh', 'Goblin', 'Griffin', 'Jonas', 'Nathair', 'Phoenix', 'Sidhe', 'Troll', 'Wyvern', 'Dochraid', 'Serket']

In [30]:
print('1: ', len(CREATURE_NAME), '2: ', len(CREATURE_CAT), '3: ', len(SEASON_EP), '4: ', len(SHOW_NAME))

1:  44 2:  44 3:  44 4:  44


In [31]:
lengths = [len(v) for v in DATABASE.values()]
print(lengths)

[44, 44, 44, 44]


In [32]:
# Save to file
pd.DataFrame(DATABASE).T.reset_index().to_csv('merlinCreatures.csv', header=True, index=False)

In [29]:
DATABASE['creature_cat'] = CREATURE_CAT